# Assignment 5: Population Analysis

## Imports

In [1]:
import pandas as pd
import numpy as np
import regex as re

## Part 1 (70%)
Write a Jupyter notebook that analyses the differences between the sexes by age in Ireland.
- Weighted mean age (by sex)
- The difference between the sexes by age

First, I read in the dataset and chose which columns I would be using

In [18]:
FILENAME="cso-populationbysex.csv"
DATADIR=r"C:\Users\ZMH\OneDrive\Desktop\PFDA\data\\" # I got a FileNotFoundError when I used "../../data/"
FULLPATH =  DATADIR + FILENAME

# Read in the CSV file
df = pd.read_csv(FULLPATH)

# Filter to only include rows where "Administrative Counties" is "Ireland"
df = df[df["Administrative Counties"] == "Ireland"]

# Create a list of columns to drop
drop_col_list = ["Statistic Label","CensusYear","Administrative Counties","UNIT"]

# Drop columns
df.drop(columns=drop_col_list, inplace=True)

# Have a look
df.head()

,Sex,Single Year of Age,VALUE
0,Male,All ages,2544549
32,Male,Under 1 year,29610
64,Male,1 year,28875
96,Male,2 years,30236
128,Male,3 years,31001


Then I removed the "All ages" from the dataset as I want to deal with each age individually.

I tidied up the ages, replacing "Under 1 year" to just "0" and also removing the "years old" from the dataset.

I also changed the ages to integers as they were strings

In [20]:
# Remove "All ages" from the dataset
df = df[df["Single Year of Age"] != "All ages"]

# I was getting errors when trying to tidy up the ages, so I changed the datatype to string first
df['Single Year of Age']=df['Single Year of Age'].astype('str')
df["VALUE"] = df["VALUE"].astype('str')

# Tidy up ages
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'Under 1 year', '0')
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'\D', '', regex=True)

# Change ages to int
df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df["VALUE"] = df["VALUE"].astype('int64')

# Show
df.head()


,Sex,Single Year of Age,VALUE
32,Male,0,29610
64,Male,1,28875
96,Male,2,30236
128,Male,3,31001
160,Male,4,31686


I then created a pivot table with the average age by sex.

Then I wrote out the cleaned up pivot tables to my computer

In [ ]:
# Create pivot table with different columns for male and female
df_pivot = pd.pivot_table(df, values="VALUE", index="Single Year of Age", columns="Sex", aggfunc="sum")
print (df_pivot.head(3))

# write out the entire file to local machine
df_pivot.to_csv("population_for_analysis.csv")

Sex                 Female   Male
Single Year of Age               
0                    28186  29610
1                    27545  28875
2                    28974  30236


Now that I have cleaned up my dataframe, I can now do some analysis.

First I want to get the weighted mean age by sex

Weighted mean is sum(age*population at age) / sum (populations at age)

In [ ]:
# Number of females
number_females = df_pivot["Female"].sum()
print(f"No. of females: {number_females}")

# Number of males
number_males = df_pivot["Male"].sum()
print(f"No. of males: {number_males}")

No. of females: 2604590
No. of males: 2544549


Part 2 20%
In the same notebook, make a variable that stores an age (say 35).

Write that code that would group the people within 5 years of that age together, into one age group 

Calculate the population difference between the sexes in that age group.

Part 3 10%
In the same notebook.

Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

End